# Brazilian E-Commerce Public Dataset by Olist

## About this file

This dataset contains detailed information about customers and their locations. It is primarily used to identify unique customers within the orders dataset and to determine the delivery location of each order.

In the Olist system, every order is assigned a distinct customer_id. As a result, the same customer may appear with different customer_ids across multiple orders. To address this, the dataset also includes a customer_unique_id field, which enables you to identify customers who have made repeat purchases. Without customer_unique_id, it would appear as though each order were linked to a completely different customer.

## Context

This dataset has been generously provided by Olist, the largest department store on Brazilian marketplaces. Olist connects small businesses from across Brazil to online sales channels seamlessly through a single contract. These merchants sell products via the Olist Store and ship directly to customers using Olist’s logistics partners.
Learn more at: www.olist.com

After a customer purchases a product from the Olist Store, the seller is notified to fulfill the order. Once the customer receives the product—or the estimated delivery date passes—the customer receives a satisfaction survey via email to rate their experience and provide feedback.

## Acknowledgements
Special thanks to Olist for making this dataset publicly available.

## QUESTION
Use appropriate machine learning model to demonstrate how machine learning can be used on e-commerce sites for product recommendations to users.

# Imports

In [4]:
# load imports
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import os

# Download the datasets from kaggle

In [6]:
# Kaggle dataset download link
url = "https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce"

cwd=os.getcwd()

# Destination folder
if 'brazilian-ecommerce' in os.listdir():
    print('The folder already exist')
    extract_dir = os.path.join(cwd, 'brazilian-ecommerce')
else:
    # Download file
    response = requests.get(url)

    # Unzip directly from memory
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(extract_dir)

The folder already exist


In [7]:
# check dataset names
datasets=os.listdir(extract_dir)
for name in datasets:
    print(name)

olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


In [8]:
# create function to open all the datasets
def open_csv(file_no):
    lst=[]
    for name in datasets:
        lst.append(name)
    df=pd.read_csv('brazilian-ecommerce/'+lst[file_no])
    return df

# Inspecting the datasets (EDA)

### 1. customers_dataset

In [11]:
customers_dataset=open_csv(0)
customers_dataset.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [12]:
# inspect nulls
customers_dataset.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [13]:
# inspect duplicates
customers_dataset[customers_dataset.duplicated()]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [14]:
# inspect data types
customers_dataset.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [15]:
# inspect categories
print(customers_dataset['customer_city'].unique())
print(customers_dataset['customer_state'].unique())

['franca' 'sao bernardo do campo' 'sao paulo' ... 'monte bonito'
 'sao rafael' 'eugenio de castro']
['SP' 'SC' 'MG' 'PR' 'RJ' 'RS' 'PA' 'GO' 'ES' 'BA' 'MA' 'MS' 'CE' 'DF'
 'RN' 'PE' 'MT' 'AM' 'AP' 'AL' 'RO' 'PB' 'TO' 'PI' 'AC' 'SE' 'RR']


In [16]:
# shape of the data
customers_dataset.shape

(99441, 5)

### 2. geolocation_dataset

In [18]:
geolocation_dataset=open_csv(1)
geolocation_dataset.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


- might be unnecessary for my question 

### 3. orders_dataset

In [21]:
orders_dataset=open_csv(2)
orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [22]:
# inspect nulls
orders_dataset.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [23]:
# inspect duplicated rows
orders_dataset[orders_dataset.duplicated()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [24]:
# inspect data types
orders_dataset.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [25]:
# inspect categories
orders_dataset['order_status'].value_counts()#unique()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [26]:
# drop orders whose status are 'unavailable' suggests stockout or system issue
available_orders =orders_dataset.loc[orders_dataset['order_status'] != 'unavailable']
available_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [27]:
# Delivered before purchase
anomaly_delivered_before_purchase = available_orders[
    available_orders['order_purchase_timestamp'] > available_orders['order_delivered_customer_date']
]
anomaly_delivered_before_purchase

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [28]:
# Carrier pickup before purchase 
anomaly_carrier_picup_before_purchase = available_orders[
    available_orders['order_purchase_timestamp'] > available_orders['order_delivered_carrier_date']
]
anomaly_carrier_picup_before_purchase[['order_purchase_timestamp', 'order_delivered_carrier_date']]

,order_purchase_timestamp,order_delivered_carrier_date
615,2018-08-16 13:50:48,2018-08-16 13:27:00
1111,2018-06-15 15:41:22,2018-06-15 14:52:00
1329,2018-05-02 10:48:44,2018-05-02 09:49:00
1372,2018-05-14 15:18:23,2018-05-14 13:46:00
1864,2018-07-26 13:25:14,2018-07-26 12:42:00
...,...,...
98172,2018-07-30 15:22:15,2018-07-30 15:00:00
98430,2018-05-04 14:50:37,2018-05-04 14:48:00
98672,2018-07-18 08:46:52,2018-07-18 08:44:00
98780,2018-07-18 16:14:16,2018-07-18 15:34:00


In [29]:
# Approval before purchase
anomaly_approval_before_purchase = available_orders[
    available_orders['order_approved_at'] < available_orders['order_purchase_timestamp']
]
anomaly_approval_before_purchase

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [30]:
# Carrier before approval
anomaly_carrier_before_approval = available_orders[
    available_orders['order_delivered_carrier_date'] < available_orders['order_approved_at']
]
anomaly_carrier_before_approval[['order_delivered_carrier_date','order_approved_at']]

,order_delivered_carrier_date,order_approved_at
15,2018-06-11 14:54:00,2018-06-12 23:31:02
64,2018-04-23 19:19:14,2018-04-24 18:25:22
199,2018-07-24 12:57:00,2018-07-26 23:31:53
210,2018-07-23 12:24:00,2018-07-23 12:31:53
415,2018-07-24 14:03:00,2018-07-27 23:31:09
...,...,...
99091,2018-07-05 14:11:00,2018-07-05 16:17:59
99230,2018-07-03 12:57:00,2018-07-05 16:32:52
99266,2018-01-31 18:11:58,2018-02-04 23:31:46
99377,2018-04-23 17:18:40,2018-04-24 19:26:10


In [31]:
# Customer delivery before carrier pickup
anomaly_delivery_before_carrier = available_orders[
    available_orders['order_delivered_customer_date'] < available_orders['order_delivered_carrier_date']
]
anomaly_delivery_before_carrier[['order_delivered_customer_date', 'order_delivered_carrier_date']]

,order_delivered_customer_date,order_delivered_carrier_date
6437,2017-07-25 19:32:56,2017-07-28 16:57:58
9553,2017-07-06 14:27:51,2017-07-07 17:22:41
13487,2017-07-19 14:13:28,2017-07-20 19:22:02
14474,2017-07-26 18:09:10,2017-08-01 18:23:30
19268,2017-07-20 18:52:41,2017-07-20 23:03:42
21338,2016-10-25 17:51:46,2016-10-26 11:41:53
22520,2017-06-26 15:45:35,2017-06-27 14:51:54
25393,2017-08-10 18:05:38,2017-08-10 18:28:56
25646,2017-07-31 17:49:56,2017-08-01 18:17:47
27470,2017-08-01 21:13:01,2017-08-09 18:18:43


In [32]:
# Delivery after estimated date
anomaly_late_delivery = available_orders[
    available_orders['order_delivered_customer_date'] > available_orders['order_estimated_delivery_date']
]
anomaly_late_delivery

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
20,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,delivered,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-10-09 22:23:46,2017-09-28 00:00:00
25,fbf9ac61453ac646ce8ad9783d7d0af6,3a874b4d4c4b6543206ff5d89287f0c3,delivered,2018-02-20 23:46:53,2018-02-22 02:30:46,2018-02-26 22:25:22,2018-03-21 22:03:54,2018-03-12 00:00:00
35,8563039e855156e48fccee4d611a3196,5f16605299d698660e0606f7eae2d2f9,delivered,2018-02-17 15:59:46,2018-02-17 16:15:34,2018-02-20 23:03:56,2018-03-20 00:59:25,2018-03-20 00:00:00
41,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,delivered,2017-11-24 21:27:48,2017-11-25 00:21:09,2017-12-13 21:14:05,2017-12-28 18:59:23,2017-12-21 00:00:00
57,66e4624ae69e7dc89bd50222b59f581f,684fa6da5134b9e4dab731e00011712d,delivered,2018-03-09 14:50:15,2018-03-09 15:40:39,2018-03-15 00:31:19,2018-04-03 13:28:46,2018-04-02 00:00:00
...,...,...,...,...,...,...,...,...
99396,b3112ca67f3afd4e20cf2ee91fc4f804,6f83c71b6c044fb156d697d4130fe9b5,delivered,2018-08-02 22:46:54,2018-08-02 23:04:06,2018-08-15 17:42:00,2018-08-21 00:03:26,2018-08-16 00:00:00
99397,0fa1fab1d7c1211c824596ed5e111e3c,7f3bd6c94d2daf7b6462d1a894a775b4,delivered,2018-03-13 21:48:57,2018-03-13 22:40:28,2018-03-14 19:27:23,2018-04-05 19:59:49,2018-04-02 00:00:00
99423,38e9133ce29f6bbe35aed9c3863dce01,ad312389a098ceff46ce92c4595c06d0,delivered,2017-10-12 20:54:11,2017-10-14 03:28:24,2017-10-17 17:04:42,2017-11-21 17:06:59,2017-10-31 00:00:00
99425,d692ef54145c9cb3322ec2e5508aa3f4,82ddfcf9438b0cd1117b55ac33184df8,delivered,2018-03-21 19:47:18,2018-03-21 20:05:26,2018-03-22 21:11:58,2018-04-11 00:48:31,2018-04-09 00:00:00


Drop the following:
- anomaly_delivery_before_carrier
- anomaly_carrier_before_approval
- anomaly_carrier_picup_before_purchase

In [34]:
anomalies=available_orders[
    (available_orders['order_purchase_timestamp'] > available_orders['order_delivered_carrier_date']) | 
    (available_orders['order_delivered_carrier_date'] < available_orders['order_approved_at']) | 
    (available_orders['order_delivered_customer_date'] < available_orders['order_delivered_carrier_date'])
]
anomalies

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
15,dcb36b511fcac050b97cd5c05de84dc3,3b6828a50ffe546942b7a473d70ac0fc,delivered,2018-06-07 19:03:12,2018-06-12 23:31:02,2018-06-11 14:54:00,2018-06-21 15:34:32,2018-07-04 00:00:00
64,688052146432ef8253587b930b01a06d,81e08b08e5ed4472008030d70327c71f,delivered,2018-04-22 08:48:13,2018-04-24 18:25:22,2018-04-23 19:19:14,2018-04-24 19:31:58,2018-05-15 00:00:00
199,58d4c4747ee059eeeb865b349b41f53a,1755fad7863475346bc6c3773fe055d3,delivered,2018-07-21 12:49:32,2018-07-26 23:31:53,2018-07-24 12:57:00,2018-07-25 23:58:19,2018-07-31 00:00:00
210,412fccb2b44a99b36714bca3fef8ad7b,c6865c523687cb3f235aa599afef1710,delivered,2018-07-22 22:30:05,2018-07-23 12:31:53,2018-07-23 12:24:00,2018-07-24 19:26:42,2018-07-31 00:00:00
415,56a4ac10a4a8f2ba7693523bb439eede,78438ba6ace7d2cb023dbbc81b083562,delivered,2018-07-22 13:04:47,2018-07-27 23:31:09,2018-07-24 14:03:00,2018-07-28 00:05:39,2018-08-06 00:00:00
...,...,...,...,...,...,...,...,...
99091,240ead1a7284667e0ec71d01f80e4d5e,fcdd7556401aaa1c980f8b67a69f95dc,delivered,2018-07-02 16:30:02,2018-07-05 16:17:59,2018-07-05 14:11:00,2018-07-10 23:21:47,2018-07-24 00:00:00
99230,78008d03bd8ef7fcf1568728b316553c,043e3254e68daf7256bda1c9c03c2286,delivered,2018-07-03 13:11:13,2018-07-05 16:32:52,2018-07-03 12:57:00,2018-07-10 17:47:39,2018-07-23 00:00:00
99266,76a948cd55bf22799753720d4545dd2d,3f20a07b28aa252d0502fe7f7eb030a9,delivered,2018-01-30 02:41:30,2018-02-04 23:31:46,2018-01-31 18:11:58,2018-03-18 20:08:50,2018-03-02 00:00:00
99377,a6bd1f93b7ff72cc348ca07f38ec4bee,6d63fa86bd2f62908ad328325799152f,delivered,2018-04-20 17:28:40,2018-04-24 19:26:10,2018-04-23 17:18:40,2018-04-28 17:38:42,2018-05-15 00:00:00


In [35]:
# Remove all the anomalies
clean_orders = available_orders.drop(anomalies.index).copy().dropna()
clean_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [36]:
# ckeck missing values
clean_orders.isna().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

In [121]:
# select the columns suited for thequestion
clean_orders=clean_orders[['order_id', 'customer_id', 'order_status']].drop_duplicates()
clean_orders

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered


### 4. ordered_items_dataset

In [39]:
order_items_dataset=open_csv(3)
order_items_dataset.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [40]:
# inspect missing values
order_items_dataset.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [41]:
# inspect duplicates
order_items_dataset[order_items_dataset.duplicated()]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


In [42]:
# inspect data types
order_items_dataset.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [43]:
# inspect data validity
order_items_dataset[['price', 'freight_value']].describe() #'shipping_limit_date' is a float

,price,freight_value
count,112650.000000,112650.000000
mean,120.653739,19.990320
std,183.633928,15.806405
min,0.850000,0.000000
25%,39.900000,13.080000
50%,74.990000,16.260000
75%,134.900000,21.150000
max,6735.000000,409.680000


In [44]:
# add price to freight_value as product price
order_items_dataset['product_price']=order_items_dataset['price']+order_items_dataset['freight_value']
order_items_dataset

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,218.04
...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,343.40
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,386.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,116.85
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,64.71


In [123]:
# select only the columns of interest
order_items_dataset=order_items_dataset[['order_id', 'order_item_id', 'product_id', 'product_price']].drop_duplicates()
order_items_dataset

,order_id,order_item_id,product_id,product_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,218.04
...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,343.40
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,386.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,116.85
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,64.71


### 5. order_payments_dataset

In [47]:
order_payments_dataset=open_csv(4)
order_payments_dataset.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [48]:
# inspect missing values
order_payments_dataset.isna().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [49]:
# inspect duplicate values
order_payments_dataset[order_payments_dataset.duplicated()]

,order_id,payment_sequential,payment_type,payment_installments,payment_value


In [50]:
# inspect data types
order_payments_dataset.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [51]:
# inspect data validity
order_payments_dataset['payment_type'].value_counts()

credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: payment_type, dtype: int64

In [52]:
order_payments_dataset[['payment_sequential', 'payment_installments', 'payment_value']].describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


In [53]:
seq_list=[]
for i in order_payments_dataset['payment_sequential'].unique():
    seq_list.append(i)
np.array(sorted(seq_list))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int64)

In [54]:
ins_list=[]
for j in order_payments_dataset['payment_installments'].unique():
    ins_list.append(j)
np.array(sorted(ins_list))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24], dtype=int64)

### 6. order_reviews_dataset

In [56]:
order_reviews_dataset=open_csv(5)
order_reviews_dataset.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [57]:
# inspect missing values
order_reviews_dataset.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [58]:
# inspect dimensionality
order_reviews_dataset.shape

(99224, 7)

In [59]:
# inspect duplicate values
order_reviews_dataset[order_reviews_dataset.duplicated()]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp


In [60]:
# inspect validity
order_reviews_dataset['review_score'].unique()

array([4, 5, 1, 3, 2], dtype=int64)

In [61]:
r_comments=[review for review in order_reviews_dataset['review_comment_message'].unique()]
pd.Series(r_comments).sample(10, random_state=42)

12592    Paguei dois produtos com 60 e só veio um na no...
31156                       produto de acordo com o preço.
34481    Nenhum dos produtos que comprei desta loja vie...
22518                 Adorei foi entregue antes do prazo !
5503     Não veio na cor conforme o anuncio, a mochila ...
8413     Muito bom o prazo de entrega veio que eu esper...
14284                porem não recebi meus produtos Ainda 
7495         Oi....veio certinho e muuuuuito rápido...amei
33586                 Bem diferente do que aparece na foto
22456    o produto foi entregue antes do prazo , produt...
dtype: object

In [62]:
# drop columns review_comment_title	and review_comment_message
order_reviews_dataset.drop(columns=['review_comment_title',	'review_comment_message'], inplace=True)
order_reviews_dataset.isna().sum()

review_id                  0
order_id                   0
review_score               0
review_creation_date       0
review_answer_timestamp    0
dtype: int64

In [63]:
order_reviews_dataset[order_reviews_dataset.duplicated()]

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp


In [129]:
order_reviews_dataset=order_reviews_dataset[['review_id', 'order_id', 'review_score']].drop_duplicates()
order_reviews_dataset

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5
...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4


### 7. products_dataset

In [191]:
products_dataset=open_csv(6)
products_dataset.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [135]:
# check columns
products_dataset.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [137]:
# select columns of interes by dropping product_name_lenght, product_description_length and product_photos_qty
products_dataset.drop(columns=['product_name_lenght', 'product_description_lenght', 'product_photos_qty'], inplace=True)
products_dataset.isna().sum()/products_dataset.shape[0]*100

product_id               0.000000
product_category_name    1.851234
product_weight_g         0.006070
product_length_cm        0.006070
product_height_cm        0.006070
product_width_cm         0.006070
dtype: float64

In [139]:
products_dataset.shape

(32951, 6)

In [195]:
# select columns
products_dataset=products_dataset[['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', \
                                   'product_height_cm', 'product_width_cm']].drop_duplicates()
products_dataset

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,700.0,31.0,13.0,20.0


In [197]:
# inspect missing values
products_dataset.dropna(inplace=True)

In [199]:
round((products_dataset.isna().sum()/products_dataset.shape[0])*100,2)

product_id               0.0
product_category_name    0.0
product_weight_g         0.0
product_length_cm        0.0
product_height_cm        0.0
product_width_cm         0.0
dtype: float64

In [201]:
# inspect duplicate values
products_dataset[products_dataset.duplicated()]

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [203]:
# inspect categories
list(pd.Series(products_dataset['product_category_name'].sample(10, random_state=2).unique()).sort_values())

['beleza_saude',
 'brinquedos',
 'cama_mesa_banho',
 'climatizacao',
 'eletrodomesticos',
 'pet_shop',
 'utilidades_domesticas']

In [205]:
products_dataset.describe()

,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32340.000000,32340.000000,32340.000000,32340.000000
mean,2276.956586,30.854545,16.958813,23.208596
std,4279.291845,16.955965,13.636115,12.078762
min,0.000000,7.000000,2.000000,6.000000
25%,300.000000,18.000000,8.000000,15.000000
50%,700.000000,25.000000,13.000000,20.000000
75%,1900.000000,38.000000,21.000000,30.000000
max,40425.000000,105.000000,105.000000,118.000000


### 8. sellers_dataset

In [76]:
sellers_dataset=open_csv(7)
sellers_dataset.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


- the seller might not be of interest for the problem

### 9. category_name_translation

In [79]:
category_name_translation=open_csv(8)
category_name_translation

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [80]:
# inspect missing values
category_name_translation.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [81]:
# inspect duplicate values
category_name_translation[category_name_translation.duplicated()]

,product_category_name,product_category_name_english


In [82]:
# create dictionary
product_dict={keys: values for keys, values in category_name_translation[['product_category_name','product_category_name_english']].values}
product_dict

{'beleza_saude': 'health_beauty',
 'informatica_acessorios': 'computers_accessories',
 'automotivo': 'auto',
 'cama_mesa_banho': 'bed_bath_table',
 'moveis_decoracao': 'furniture_decor',
 'esporte_lazer': 'sports_leisure',
 'perfumaria': 'perfumery',
 'utilidades_domesticas': 'housewares',
 'telefonia': 'telephony',
 'relogios_presentes': 'watches_gifts',
 'alimentos_bebidas': 'food_drink',
 'bebes': 'baby',
 'papelaria': 'stationery',
 'tablets_impressao_imagem': 'tablets_printing_image',
 'brinquedos': 'toys',
 'telefonia_fixa': 'fixed_telephony',
 'ferramentas_jardim': 'garden_tools',
 'fashion_bolsas_e_acessorios': 'fashion_bags_accessories',
 'eletroportateis': 'small_appliances',
 'consoles_games': 'consoles_games',
 'audio': 'audio',
 'fashion_calcados': 'fashion_shoes',
 'cool_stuff': 'cool_stuff',
 'malas_acessorios': 'luggage_accessories',
 'climatizacao': 'air_conditioning',
 'construcao_ferramentas_construcao': 'construction_tools_construction',
 'moveis_cozinha_area_de_ser

### Merging the data sets

In [215]:
import pandas as pd

# aggregate payments per order
payments_agg = (order_payments_dataset
                .groupby('order_id')
                .agg({'payment_value':'sum',
                      'payment_type': lambda x: ','.join(x.unique())})
                .reset_index())

# aggregate items per order
items_agg = (order_items_dataset
             .groupby('order_id')
             .agg({'product_id': lambda x: ','.join(x),
                   'product_price':'sum'})
             .reset_index())

orders = (clean_orders
          .merge(order_reviews_dataset, on='order_id', how='left')
          .merge(payments_agg, on='order_id', how='left')
          .merge(items_agg, on='order_id', how='left')
          .merge(customers_dataset, on='customer_id', how='left'))


In [217]:
orders

,order_id,customer_id,order_status,review_id,review_score,payment_value,payment_type,product_id,product_price,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4.0,38.71,"credit_card,voucher",87285b34884572647811a353c7ac498a,38.71,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,8d5266042046a06655c8db133d120ba5,4.0,141.46,boleto,595fac2a385ac33a80bd5114aec74eb8,141.46,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,e73b67b67587f7644d5bd1a52deb1b01,5.0,179.12,credit_card,aa4383b373c6aca5d8797843e5594415,179.12,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,359d03e676b3c069f62cadba8dd3f6e8,5.0,72.20,credit_card,d0b61bfb1de832b15ba9d266ca96e5b0,72.20,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,e50934924e227544ba8246aeb3770dd4,5.0,28.62,credit_card,65266b2da20d04dbe00c5c2d3bb7859e,28.62,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95610,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,e262b3f92d1ce917aa412a9406cf61a6,5.0,85.08,credit_card,ac35486adb7b02598c182c2ff2e05254,85.08,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP
95611,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,29bb71b2760d0f876dfa178a76bc4734,4.0,195.00,credit_card,f1d4ce8c6dd66c47bbaa8c6781c2a923,195.00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP
95612,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,371579771219f6db2d830d50805977bb,5.0,271.01,credit_card,b80910977a37536adeddd63663f916ad,271.01,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA
95613,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,8ab6855b9fe9b812cd03a480a25058a1,2.0,441.16,credit_card,"d1c427060a0f73f6b889a5c7c61f2ac4,d1c427060a0f7...",441.16,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ


In [219]:
# orders=all orders on order_id
orders=(clean_orders
    .merge(order_reviews_dataset, on='order_id', how='inner')
    .merge(order_payments_dataset, on='order_id', how='inner')
    .merge(order_items_dataset, on ='order_id', how='inner')
    .merge(products_dataset, on ='product_id', how='inner')
    .merge(customers_dataset, on='customer_id', how='inner')
)
orders.head(20)


,order_id,customer_id,order_status,review_id,review_score,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,...,product_price,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4,1,credit_card,1,18.12,1,...,38.71,utilidades_domesticas,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4,3,voucher,1,2.00,1,...,38.71,utilidades_domesticas,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4,2,voucher,1,18.59,1,...,38.71,utilidades_domesticas,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,b46f1e34512b0f4c74a72398b03ca788,4,1,credit_card,3,37.77,1,...,37.77,utilidades_domesticas,500.0,19.0,8.0,13.0,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,dc90f19c2806f1abba9e72ad3c350073,5,1,credit_card,1,37.77,1,...,37.77,utilidades_domesticas,500.0,19.0,8.0,13.0,ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP
5,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,1bafb430e498b939f258b9c9dbdff9b1,3,1,boleto,1,44.09,1,...,44.09,utilidades_domesticas,500.0,19.0,8.0,13.0,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC
6,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,8d5266042046a06655c8db133d120ba5,4,1,boleto,1,141.46,1,...,141.46,perfumaria,400.0,19.0,13.0,19.0,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
7,40c5e18f7d112b59b3e5113a59a905b3,67407057a7d5ee17d1cd09523f484d13,delivered,87bed0b5fb6856c9328d5e60df70546a,5,1,credit_card,3,128.68,1,...,128.68,perfumaria,400.0,19.0,13.0,19.0,7cfba6e55439cae3fd2479d62fafe67f,22240,rio de janeiro,RJ
8,86674ccaee19790309333210917b2c7d,1b338293f35549b5e480b9a3d7bbf3cd,delivered,ebd3c6f3c54ee7cbdecefd57d5d57d1e,5,1,credit_card,5,135.82,1,...,135.82,perfumaria,400.0,19.0,13.0,19.0,8b8028d453a01986b48c4271cc341475,14500,ituverava,SP
9,f913d229653fdd809c249ed98ab6b754,e1365d7b227b247b6bc0931771885eaf,delivered,ff25eee85259b7a0991f9643a0b6dc34,5,1,credit_card,4,137.45,1,...,137.45,perfumaria,400.0,19.0,13.0,19.0,66c70ed7726abccfbee61a9a39de15c5,90050,porto alegre,RS


In [209]:
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'review_id', 'review_score',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'order_item_id', 'product_id', 'product_price',
       'product_category_name', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state'],
      dtype='object')

In [85]:
# customers and sellers geo
customers_geo = customers_dataset.merge(geolocation_dataset, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left'
sellers_geo = sellers_dataset.merge(geolocation_dataset, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

SyntaxError: invalid syntax (3991004892.py, line 3)

In [ ]:
# create customers geographical data
geo_grouped = geolocation_dataset.groupby("geolocation_zip_code_prefix")[["geolocation_lat", "geolocation_lng"]].mean().reset_index()

customers_geo = customers_dataset.merge(
    geo_grouped,
    left_on="customer_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left"
)
customers_geo.sample(10, random_state=3)

In [ ]:
# create sellers geographical data
geo_grouped = geolocation_dataset.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

sellers_geo = sellers_dataset.merge(
    geo_grouped,
    left_on='seller_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)
sellers_geo.sample(10, random_state=5)

In [ ]:
# merge orders to customers and sellers
ecommerce = (orders
    .merge(customers_geo, on='customer_id', how='inner')
    .merge(sellers_geo, on='seller_id', how='inner')
)
ecommerce.head()

In [ ]:
# full dataset merged
brazilian_ecommerce = (ecommerce
    .merge(products_dataset, on ='product_id', how='inner')
)
brazilian_ecommerce.head()

- For this recommendations, we need customer, product and how customers interact with the products i.e., purchase, rating

In [ ]:
columns=['customer_unique_id', 'product_id', 'order_id', 'seller_id',\
          'customer_state', 'seller_state', \
         'review_score',  'product_category_name', 'price', \
                'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
brazilian_ecommerce = brazilian_ecommerce[columns]
brazilian_ecommerce.isna().sum()

In [ ]:
# rename the product_category_name to english
brazilian_ecommerce['product_category_name'] = brazilian_ecommerce['product_category_name'].map(product_dict)
brazilian_ecommerce['product_category_name'].unique()

In [ ]:
brazilian_ecommerce.drop_duplicates(inplace=True)

In [ ]:
brazilian_ecommerce.shape

In [ ]:
brazilian_ecommerce_clean = brazilian_ecommerce.copy()
brazilian_ecommerce_clean

In [ ]:
brazilian_ecommerce_clean.dtypes

In [ ]:
# coulmns
brazilian_ecommerce_clean.describe()

# Collaborative Filtering (CF)


In [ ]:
# required columns for CF
cf_data=brazilian_ecommerce_clean[['customer_unique_id', 'product_id', 'review_score']].drop_duplicates()
cf_data

### (a) Memory-Based CF

Memory-based CF → works directly on the user–item matrix (cosine similarity, Pearson correlation, kNN).

Model-based CF → uses factorization models (SVD, SVD++, NMF, etc.) or even neural models.

Memory-based CF (similarity on sparse matrix)

Here you need a user–item matrix, but keep it sparse (not a dense pivot).


In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [ ]:
# encode ids to numeric
user_ids = cf_data['customer_unique_id'].astype('category').cat.codes
item_ids = cf_data['product_id'].astype('category').cat.codes

In [ ]:
# build sparse matrix
user_item_matrix = csr_matrix((cf_data['review_score'], (user_ids, item_ids)))

User-based CF (find similar users)

Instead of computing a full similarity matrix (too big), use Nearest Neighbors.

In [ ]:
# Train kNN on users
user_knn = NearestNeighbors(metric='cosine', algorithm='brute')
user_knn.fit(user_item_matrix)

# Example: find top-5 similar users for user 0
distances, indices = user_knn.kneighbors(user_item_matrix[0], n_neighbors=5)
print(indices)

In [ ]:
# Item-based CF (find similar items)-Recommend items similar to what the user already liked.
item_knn = NearestNeighbors(metric='cosine', algorithm='brute')
item_knn.fit(user_item_matrix.T)

# Example: find top-5 similar items for product 0
distances, indices = item_knn.kneighbors(user_item_matrix.T[0], n_neighbors=5)
print(indices)

Model-based Collaborative Filtering

This learns latent features of users & items using matrix factorization (SVD, NMF, etc.).

We’ll use the Surprise library.

In [ ]:
from surprise import Dataset, Reader, SVD, NMF, KNNBasic
from surprise.model_selection import cross_validate

In [ ]:
reader = Reader(rating_scale=(1, 5))  # reviews are 1–5
data = Dataset.load_from_df(cf_data[['customer_unique_id', 'product_id', 'review_score']], reader)

In [ ]:
# Train a model (SVD as example)
algo = SVD()

# Evaluate with cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
# Train on all data and make predictions
trainset = data.build_full_trainset()
algo.fit(trainset)

# Predict rating: user_id, product_id
pred = algo.predict('7c396fd4830fd04220f754e42b4e5bff', '87285b34884572647811a353c7ac498a')
print(pred.est)  # estimated rating

In [ ]:
# Recommend top-N items to a user
def recommend_top_n(algo, user_id, cf_data, n=5):
    all_items = cf_data['product_id'].unique()
    rated_items = cf_data[cf_data['customer_unique_id'] == user_id]['product_id'].unique()
    
    # Candidate items = not yet rated
    candidates = [item for item in all_items if item not in rated_items]
    
    # Predict ratings
    predictions = [algo.predict(user_id, item) for item in candidates]
    
    # Sort by estimated rating
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return [(pred.iid, pred.est) for pred in top_n]

recommendations = recommend_top_n(algo, '7c396fd4830fd04220f754e42b4e5bff', cf_data, n=5)
print(recommendations)


### Model Evaluation
RMSE / MAE → prediction accuracy (how close predicted ratings are to true ratings).
Precision@K / Recall@K → recommendation quality (how many good recommendations are in the top-K list)

In [ ]:
# Model-based CF Evaluation (with Surprise)

# Load dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(cf_data[['customer_unique_id', 'product_id', 'review_score']], reader)

# Model-based: SVD
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Model-based: NMF (Non-negative Matrix Factorization)
nmf = NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
#Memory-based CF Evaluation (with Surprise KNN)

# User-based CF
sim_options = {'name': 'cosine', 'user_based': True}
knn_user = KNNBasic(sim_options=sim_options)
#cross_validate(knn_user, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Item-based CF
sim_options = {'name': 'cosine', 'user_based': False}
knn_item = KNNBasic(sim_options=sim_options)
cross_validate(knn_item, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
#Ranking Evaluation (Precision@K & Recall@K)

#RMSE is nice, but what you really care about is:“When I recommend top-K items, are they good?”

#Surprise doesn’t include ranking metrics by default, but you can implement them:

from collections import defaultdict
from surprise.model_selection import train_test_split

def get_top_n(predictions, n=10):
    # Map user -> predictions
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Keep top-n for each user
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Train/test split
trainset, testset = train_test_split(data, test_size=0.25)

# Example with SVD
svd.fit(trainset)
predictions = svd.test(testset)

# Top-10 recommendations per user
top_n = get_top_n(predictions, n=10)

# Show sample recommendations
for uid, user_ratings in list(top_n.items())[:5]:
    print(uid, [iid for (iid, _) in user_ratings])

Hybrid Recommendation

Feature Augmentation (Model-based Hybrid)

Add content features into the CF model.

Example: Instead of only using user_id, item_id, and review_score, also add side features (category, price, weight, dimensions).

Tool: LightFM (very good for this, built for hybrid recommenders).

👉 Example with LightFM:

Ways to Improve

Better algorithms

Try SVDpp, NMF, or KNNBaseline (from surprise) and compare.

SVD++ often improves accuracy for implicit feedback.

Hyperparameter tuning

Use GridSearchCV on n_factors, reg_all, lr_all.

Example: increase n_factors (latent features) from 50 → 200.

Data filtering

Remove users/items with too few ratings (e.g., < 3).

Helps reduce noise.